In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
df_ = pd.read_csv("archive/son1.6.csv")
df_.drop('Unnamed: 0', axis=1, inplace=True)
df = df_.copy()

In [41]:
df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,...,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,26196,2,0,419136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",73194,3,0,219582
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,1462,2,0,68714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,9575775,5,45,-383031
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22368,2,2,25164


In [4]:
# Tarih ve satış sütunlarının düzenlenmesi
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors='coerce')
df['Sales'] = df['Sales'].replace(',', '.', regex=True).astype(float)

In [5]:
df1 = df.groupby(["Customer_ID"]).agg({"Order_Date":["min","max"],
                              "Order_ID":"count",
                             "Sales":"sum",
                             })

In [6]:
df.describe()

,Row_ID,Order_Date,Postal_Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994,9994.000000,9.994000e+03,9994.000000,9994.000000,9.994000e+03
mean,4997.500000,2016-04-30 00:07:12.259355648,55190.379428,1.132271e+05,3.789574,1.754653,1.800957e+05
min,1.000000,2014-01-03 00:00:00,1040.000000,3.000000e+00,1.000000,0.000000,-3.839990e+07
25%,2499.250000,2015-05-23 00:00:00,23223.000000,2.516000e+03,2.000000,0.000000,2.245000e+03
50%,4997.500000,2016-06-26 00:00:00,56430.500000,1.108800e+04,3.000000,2.000000,3.630900e+04
75%,7495.750000,2017-05-14 00:00:00,90008.000000,5.399175e+04,5.000000,2.000000,1.509840e+05
max,9994.000000,2017-12-30 00:00:00,99301.000000,2.396266e+07,14.000000,45.000000,6.719981e+07
std,2885.163629,NaN,32063.693350,5.186212e+05,2.225110,3.119014,1.722971e+06


In [7]:
df1.columns = ['_'.join(col).strip() for col in df1.columns]

In [8]:
df1 = df1.reset_index()
df1.head()

,Customer_ID,Order_Date_min,Order_Date_max,Order_ID_count,Sales_sum
0,AA-10315,2014-03-31,2017-06-29,11,5090484.0
1,AA-10375,2014-04-21,2017-12-11,15,165678.0
2,AA-10480,2014-05-04,2017-04-15,12,193048.0
3,AA-10645,2014-06-22,2017-11-05,18,1737756.0
4,AB-10015,2014-02-18,2016-11-10,6,143818.0


In [9]:
reference_date = pd.to_datetime('2018-1-09')

In [10]:
df1["Recency"] = (reference_date - df1["Order_Date_max"]).dt.days

In [11]:
df1["Tenure"] = (reference_date - df1["Order_Date_min"]).dt.days

In [12]:
df1.head()

,Customer_ID,Order_Date_min,Order_Date_max,Order_ID_count,Sales_sum,Recency,Tenure
0,AA-10315,2014-03-31,2017-06-29,11,5090484.0,194,1380
1,AA-10375,2014-04-21,2017-12-11,15,165678.0,29,1359
2,AA-10480,2014-05-04,2017-04-15,12,193048.0,269,1346
3,AA-10645,2014-06-22,2017-11-05,18,1737756.0,65,1297
4,AB-10015,2014-02-18,2016-11-10,6,143818.0,425,1421


In [13]:
df1["Frequency"] = df1["Order_ID_count"]

In [14]:
df1["Monetary"] = df1["Sales_sum"]

In [15]:
df1["Basket_Size"] = df1["Sales_sum"]/df1["Frequency"]

In [16]:
df1 = df1.drop(["Order_Date_min","Order_Date_max","Order_ID_count","Order_ID_count"], axis = 1)

In [17]:
df1.index = df1["Customer_ID"]

In [18]:
df1.drop("Customer_ID", axis=1, inplace=True)

In [19]:
df1.head()

,Sales_sum,Recency,Tenure,Frequency,Monetary,Basket_Size
Customer_ID,,,,,,
AA-10315,5090484.0,194,1380,11,5090484.0,462771.272727
AA-10375,165678.0,29,1359,15,165678.0,11045.200000
AA-10480,193048.0,269,1346,12,193048.0,16087.333333
AA-10645,1737756.0,65,1297,18,1737756.0,96542.000000
AB-10015,143818.0,425,1421,6,143818.0,23969.666667


In [20]:
df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,...,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,26196.0,2,0,419136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",73194.0,3,0,219582
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,1462.0,2,0,68714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,9575775.0,5,45,-383031
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22368.0,2,2,25164


In [21]:
min(df["Order_Date"])

Timestamp('2014-01-03 00:00:00')

In [22]:
max(df["Order_Date"])

Timestamp('2017-12-30 00:00:00')

In [24]:
datediff = max(df["Order_Date"]) - min(df["Order_Date"])
print(datediff)

1457 days 00:00:00


In [25]:
df["Quantity"].count()

9994

In [30]:
type(datediff.days)

int

In [31]:
df["Quantity"].count()/datediff.days

6.85929993136582

In [35]:
sortValues = df.groupby("Order_Date").agg({"Quantity":"count"}).sort_values(by="Quantity", ascending=False)

In [37]:
sortValues["Quantity"].mean()

8.079223928860145